# Description

This file is a new version of the ant simulation that is in Simulation.ipynb. It tries to simplify and optimize the code.

It is a simulation of ant behavior based on the following model:

Ontology:
- Ants
    - have a position vector that determines their location
    - have a direction defined by an angle theta
    - have a speed
- Pheromone
    - it is a gas
    - there is a concentration of this gas at every point of the environment/surface

Dynamics:
- Ants
    - at every time step, the ant either moves inertially, randomly, or towards the gradient:
        - if they move inertially, they move at their speed in the direction theta
        - if they move towards the gradient, they find the direction between their immediate neighbors where the pheromone concentration is maximum and move 'towards' that direction; their speed is also updated depending on the concentration of the pheromone)
        - if they move randomly, they pick a random neighbor and move towards that direction
    - if after moving they would hit the wall/boundary, they pick a random point in the confines of the surface and move in that direction
    - if they after moving they would hit another ant, they stop where they are and change their theta by a random amount in [-pi/2,pi/2]
- Pheromone
    - at every time step, the pheromone diffuses over the surface and evaporates
    - the pheromone cannot diffuse across the boundary, so it collects at the boundary
    - the exact evolution is determines as P_new = (1 - E) * (P_old + (D * (P_ave - P_old))),
        - E is the rate of evaporation in [0,1]
        - D is the rate of diffusion in [0,1]
        - P_new is the new/evolved concentration of pheromone at a point in the surface
        - P_old is the old concentration
        - P_ave is the average of the old concentrations in a point and its immediate neighbors (9 total points)

Notes about Discrete vs Continuous Quantities:

The model that has just been described deals with continuous variables (an ants angle, speed, and position are all continuous). However, we necessarily have to deal with discrete quantities when visualizing the simulation. Moreovoer, the pheromone, though clearly continuous in real life, is best described discretely in a model, as a 2D array. Therefore, the ant position will need to be visualized discretely, even though the underlying quantities are continuous.

# Setup

In [2]:
import pygame
from pygame.math import Vector2 as Vector
import numpy as np
import math
import random
import copy
import os

pygame 2.5.2 (SDL 2.28.3, Python 3.7.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Some Independent Functions

In [3]:
def get_grid_neighbors(x,y):
    neighbors = []
    for x2 in range(x-1, x+2):
        for y2 in range(y-1, y+2):
            if (
                -1 < x <= SIM_WIDTH and -1 < y <= SIM_HEIGHT 
                and (x != x2 or y != y2) 
                and (0 <= x2 <= SIM_WIDTH) and (0 <= y2 <= SIM_HEIGHT)
               ):
                neighbors.append((x2,y2))
    return neighbors

# Define a function to map pheromone potential to color
def potential_to_color(potential, vmin, vmax):
    # Normalize the potential value
    normalized_value = (potential - vmin) / (vmax - vmin)
    
    # Map the normalized value to a color between light yellow (255, 255, 204) and dark red (189, 0, 38)
    color = (
        int(255 - 66 * normalized_value),
        int(255 - 255 * normalized_value),
        int(204 - 166 * normalized_value))
    
    return color

## Ant Class

In [116]:
# ANT CONSTANTS
ANT_RADIUS = 1
ANT_LENGTH = 2
MIN_SPEED = 0.5
MAX_SPEED = 2
K_P = 0.75
SPEED_ALPHA = 0.5
P_FORAGING = 0
P_HOMING = 1 - P_FORAGING
MIN_PH_SENS = 0.1
THETA_STOCHASTICITY = np.pi/8

class ANT:
    def __init__(self,x,y,num):
        self.num = num
        self.pos = Vector(x,y)
        self.theta = random.uniform(0,2*np.pi)
        self.speed = random.uniform(MIN_SPEED,MAX_SPEED)
        if random.uniform(0,1) <= P_FORAGING or self.num == 5:
            self.mode = "FORAGING"
        else:
            self.mode = "HOMING"
    
    def draw_ant(self,surface):
        x_coord = round(self.pos.x) * CELL_SIZE
        y_coord = round(self.pos.y) * CELL_SIZE
        x_end_coord = round(self.pos.x - ANT_LENGTH*math.cos(self.theta)) * CELL_SIZE
        y_end_coord = round(self.pos.y - ANT_LENGTH*math.sin(self.theta)) * CELL_SIZE
        ant_rect = pygame.Rect(x_coord, y_coord, ANT_RADIUS*CELL_SIZE, ANT_RADIUS*CELL_SIZE)
        pygame.draw.rect(surface,'black',ant_rect)
#         pygame.draw.line(surface, (0,0,0), (x_coord,y_coord), (x_end_coord,y_end_coord), 3)
        
    def avoid_wall(self):
        if round(self.pos.x) <= (1 + ANT_RADIUS) or round(self.pos.x) >= (SIM_WIDTH - 1 - ANT_RADIUS) \
            or int(self.pos.y) <= (ANT_RADIUS + 1) or round(self.pos.y) >= (SIM_HEIGHT - 1 - ANT_RADIUS):
            center = (round(random.uniform(0.1*SIM_WIDTH,0.9*SIM_WIDTH)),round(random.uniform(0.1*SIM_HEIGHT,0.9*SIM_HEIGHT)))
            direction = (center[0] - round(self.pos.x), center[1] - round(self.pos.y))
            self.theta = math.atan2(direction[1], direction[0])
    
    def avoid_collision(self, ants):
        xdot = self.speed * np.cos(self.theta)
        ydot = self.speed * np.sin(self.theta)

        new_x_coord = round(self.pos.x + xdot)
        new_y_coord = round(self.pos.y + ydot)
        for ant in ants:
            if abs(new_x_coord - round(ant.pos.x)) == 0 and abs(new_y_coord - round(ant.pos.y)) == 0:
                self.theta += random.uniform(-np.pi/16,np.pi/16)
                return False
        return True
    
    # Basic function to find gradient by looking for neighbor with highest concentration of pheromone.
    def find_gradient_direction_1(self,pheromone_field):
        x, y = round(self.pos.x), round(self.pos.y)
        x, y = min(max(x,0),SIM_WIDTH), min(max(y,0),SIM_HEIGHT)
        neighbors = get_grid_neighbors(x,y)
        maximum_pheromone = MIN_PH_SENS * VMAX #potential_field[location_x][location_y]
        max_pheromone_neighbors = [(x,y)]
        for n in neighbors:
            n_x, n_y = n
            if pheromone_field[n_x][n_y] > 1.1 * maximum_pheromone:
                maximum_pheromone = pheromone_field[n_x][n_y]
                max_pheromone_neighbors = [(n_x,n_y)]
            elif pheromone_field[n_x][n_y] > 0.9 * maximum_pheromone:
                max_pheromone_neighbors.append((n_x,n_y))
        max_pheromone_neighbor = random.choice(max_pheromone_neighbors)

        return max_pheromone_neighbor, maximum_pheromone
    
    # More complex function to calculate the gradient that uses finite differences.
    def find_gradient_direction_2(self,grid):
        # Get approximate ant location to find gradient.
        x, y = round(self.pos.x), round(self.pos.y)
        x, y = min(max(x,0),SIM_WIDTH), min(max(y,0),SIM_HEIGHT)
        
        # Calculate dimensions of pheromone field.
        width = SIM_WIDTH
        height = SIM_HEIGHT
    
        gradient_x = 0
        gradient_y = 0

        # Compute gradient using central finite differences
        if 0 < y < height - 1:
            gradient_y = (grid[x][y+1] - grid[x][y-1]) / 2.0
        elif y == 0:
            gradient_y = grid[x][y+1] / 2.0
        else:
            gradient_y = -grid[x][y-1] / 2.0
        if 0 < x < width - 1:
            gradient_x = (grid[x+1][y] - grid[x-1][y]) / 2.0
        elif x == 0:
            gradient_x = grid[x+1][y] / 2.0
        else:
            gradient_x = -grid[x-1][y] / 2.0

        # Add contributions from diagonal neighbors
        if 0 < x < width - 1 and 0 < y < height - 1:
            gradient_x += (grid[x+1][y+1] - grid[x-1][y-1]) / (2.0 * np.sqrt(2))
            gradient_y += (grid[x+1][y-1] - grid[x-1][y+1]) / (2.0 * np.sqrt(2))
        elif x == 0 and y == 0:
            gradient_x += (grid[0][1] - grid[0][0]) / 2.0
            gradient_y += (grid[1][0] - grid[0][0]) / 2.0
        elif x == 0 and y == height - 1:
            gradient_x += (grid[0][height - 1] - grid[0][height - 2]) / 2.0
            gradient_y += (grid[1][height - 1] - grid[0][height - 1]) / 2.0
        elif x == width - 1 and y == 0:
            gradient_x += (grid[rows - 1][1] - grid[width - 1][0]) / 2.0
            gradient_y += (grid[rows - 1][0] - grid[width - 2][0]) / 2.0
        elif x == width - 1 and y == height - 1:
            gradient_x += (grid[rows - 1][cols - 1] - grid[width - 1][height - 2]) / 2.0
            gradient_y += (grid[rows - 1][cols - 1] - grid[width - 2][height - 1]) / 2.0

        # Compute gradient magnitude
        gradient_magnitude = np.sqrt(gradient_x ** 2 + gradient_y ** 2)

        return [gradient_x, gradient_y], gradient_magnitude
    
    def move_ant(self, pheromone_field, ants):
        if self.mode == "FORAGING":
            self.theta += random.uniform(-THETA_STOCHASTICITY,THETA_STOCHASTICITY)
            self.avoid_wall()
            
            self.speed = max(self.speed, (MIN_SPEED + MAX_SPEED) / 2)
            
            xdot = self.speed * np.cos(self.theta)
            ydot = self.speed * np.sin(self.theta)
            
            move = self.avoid_collision(ants)
            if move:
                self.pos.x += xdot
                self.pos.y += ydot
        
        elif self.mode == "HOMING":
            gradient_direction, gradient_V = self.find_gradient_direction_1(pheromone_field)
            gradient = (gradient_direction[0] - round(self.pos.x), gradient_direction[1] - round(self.pos.y))
            if gradient[0] != 0:
                gradient_theta = math.atan2(gradient[1], gradient[0])
            elif gradient[1] == 0:
                gradient_theta = self.theta + random.uniform(-THETA_STOCHASTICITY,THETA_STOCHASTICITY)
            else:
                gradient_theta = (np.pi/2)*(gradient[1] - round(self.pos.y))

            thetadot = K_P*np.sin(gradient_theta - self.theta)
            self.theta += thetadot

            self.avoid_wall()
            
            # slow down or speed up
            bounded_gradient_V = min(max(0,gradient_V),VMAX)
            target_speed = MAX_SPEED * (1 - bounded_gradient_V / VMAX)
            self.speed = self.speed + SPEED_ALPHA * (target_speed - self.speed)
            self.speed = min(max(MIN_SPEED,self.speed),MAX_SPEED)
            
            xdot = self.speed * np.cos(self.theta)
            ydot = self.speed * np.sin(self.theta)
            
            move = self.avoid_collision(ants)
            if move:
                self.pos.x += xdot
                self.pos.y += ydot

## Simulation Class

In [124]:
# PHEROMONE CONSTANTS
D = 0.25 #0.2
E = 0.001 #0.001
VMIN = 0
VMAX = 100
M = 10
P_DROP = 0.5
PHEROMONE_INTERVAL = 1

class SIM:
    def __init__(self,DIMS,NUM_ANTS):
        width, height = DIMS
        a_per_line = math.floor(math.sqrt(NUM_ANTS))
        a_spacing = math.floor(width / (a_per_line+1))
        self.pheromone_timer = 0
        self.pheromone_field = [[0 for i in range(width+1)] for j in range(height+1)]
        self.ants = [ANT(x * a_spacing, y * a_spacing, y + (x-1)*a_per_line) for x in range(1,a_per_line+1) for y in range(1,a_per_line+1)]
    
    def update(self):
        self.diffuse_pheromone()
        self.drop_pheromone()
        for ant in self.ants:
            ant.move_ant(self.pheromone_field, self.ants)
    
    def draw_elements(self,surface,display):
        # fill the screen with white
        screen.fill((255, 255, 255))
        
        # draw pheromone field
        self.draw_pheromone(surface)
        
        # draw ants
        for ant in self.ants:
            ant.draw_ant(surface)
        
        if display:
            pygame.display.update()
    
    def draw_pheromone(self,surface):
        p_width = len(self.pheromone_field[0])
        p_height = len(self.pheromone_field)
        
        for x in range(p_width):
            for y in range(p_height):
                concentration = self.pheromone_field[x][y]
                bounded_concentration = min(max(concentration,VMIN),VMAX)
                color = potential_to_color(bounded_concentration, VMIN, VMAX)
                
                x_coord, y_coord = int(x * CELL_SIZE), int(y * CELL_SIZE)
                # Draw a rectangle at the current point with the calculated color
                p_rect = pygame.Rect(x_coord, y_coord, CELL_SIZE, CELL_SIZE)
                pygame.draw.rect(surface, color, p_rect)
    
    def drop_pheromone(self):
        if self.pheromone_timer >= PHEROMONE_INTERVAL:
            for ant in self.ants:
                if ant.num != -1:
                    if random.uniform(0,1) < P_DROP:
                        # Determine ant coordinates.
                        drop_x, drop_y = round(ant.pos.x), round(ant.pos.y)
                        # Bound coordinates to within simulation dimensions.
                        drop_x, drop_y = max(min(drop_x,SIM_WIDTH-1),1), max(min(drop_y,SIM_HEIGHT-1),1)
                        # Add drop.
                        self.pheromone_field[drop_x][drop_y] = VMAX * M
            # Reset timer.
            self.pheromone_timer = 0
    
    def diffuse_pheromone(self):
        p_width = len(self.pheromone_field[0])
        p_height = len(self.pheromone_field)
    
        old_field = copy.deepcopy(self.pheromone_field)
        for x in range(1,p_width-1):
            for y in range(1,p_height-1):
                square_concentration = old_field[x][y]
                neighbors = get_grid_neighbors(x,y)
                total_pheromone = square_concentration
                for n in neighbors:
                    total_pheromone += old_field[n[0]][n[1]]
                ave_pheromone = total_pheromone / (len(neighbors) + 1)
                next_concentration = (1 - E) * (square_concentration + (D * (ave_pheromone - square_concentration)))
#                 next_concentration = min(max(VMIN,next_concentration),VMAX)
                self.pheromone_field[x][y] = next_concentration

## Run and Visualize Simulation

In [1]:
# User-set parameters.
display = True
record = False
save_ant_locations = True
if save_ant_locations: 
    ant_locations = []

# Output directory for saving frames
OUTPUT_DIR = "frames1"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# video making constants and parameters
DESIRED_DURATION = 60 # Desired duration in seconds
FPS = 24
desired_frames = int(FPS * DESIRED_DURATION)
frame_counter = 0

# simulation and visualization dimensions
SIM_WIDTH, SIM_HEIGHT = 100, 100
CELL_SIZE = 5
VIS_WIDTH, VIS_HEIGHT = CELL_SIZE*SIM_WIDTH, CELL_SIZE*SIM_HEIGHT

NUM_ANTS = 10

# initialize simulation
simulation = SIM((SIM_WIDTH,SIM_HEIGHT),NUM_ANTS)

# append initial conditions to ant locations
if save_ant_locations: 
    current_ant_locations = []
    for ant in simulation.ants:
        current_ant_locations.append((ant.pos.x,ant.pos.y))
    ant_locations.append(current_ant_locations)

# initialize visualization screen and simulation surface
pygame.init()
screen = pygame.display.set_mode((VIS_WIDTH,VIS_HEIGHT))
pygame.display.set_caption("Scaled Simulation")

# set up time objects and variables
clock = pygame.time.Clock()

# loop to display simulation
running = True
while running:
    for event in pygame.event.get():
        # Click SPACE to end simulation.
        if event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
            running = False
        # Click on the screen to add pheromone.
        elif event.type == pygame.MOUSEBUTTONDOWN:
            # Check if the left mouse button was clicked
            if event.button == 1:
                mouse_x, mouse_y = event.pos
                drop_x, drop_y = round(mouse_x / CELL_SIZE), round(mouse_y / CELL_SIZE)
                drop_x, drop_y = max(min(drop_x,SIM_WIDTH-1),1), max(min(drop_y,SIM_HEIGHT-1),1)
                simulation.pheromone_field[drop_x][drop_y] += VMAX*M
        # Click right arrow to start/stop recording simulation.
        elif event.type == pygame.KEYDOWN and event.key == pygame.K_RIGHT:
            record = bool( (record + 1) % 2 )
        
    simulation.draw_elements(screen,display)
    
    if record:
        frame_filename = os.path.join(OUTPUT_DIR, f"frame_{frame_counter:04d}.png")
        pygame.image.save(screen, frame_filename)
        frame_counter += 1
    
    simulation.pheromone_timer += clock.get_time() / 1000.0
    simulation.update()
    
    if save_ant_locations:
        current_ant_locations = []
        for ant in simulation.ants:
            current_ant_locations.append((ant.pos.x,ant.pos.y))
        ant_locations.append(current_ant_locations)
    
    clock.tick(60)

pygame.quit()

NameError: name 'os' is not defined

## Visualize Ant Trajectories

In [126]:
# simulation and visualization dimensions
SIM_WIDTH, SIM_HEIGHT = 100, 100
CELL_SIZE = 5
VIS_WIDTH, VIS_HEIGHT = CELL_SIZE*SIM_WIDTH, CELL_SIZE*SIM_HEIGHT

pygame.init()
screen = pygame.display.set_mode((VIS_WIDTH,VIS_HEIGHT))
pygame.display.set_caption("Scaled Simulation")

simulation.draw_elements(screen,display)

ant_locations_transposed = [[(x * CELL_SIZE, y * CELL_SIZE) for x, y in row] for row in zip(*ant_locations)]

for ant in ant_locations_transposed:
    pygame.draw.lines(screen, (random.uniform(0,200),random.uniform(0,200),random.uniform(0,200)), False, ant)
#         pygame.draw.rect(screen,'blue',pygame.Rect(round(position[0])*CELL_SIZE, round(position[1])*CELL_SIZE, 1, 1))

pygame.display.update()

### 